<a href="https://colab.research.google.com/github/fayrozfarouk/unsupervised_cnn_clustering/blob/master/fay_unsupervised_cnn_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.13

import pandas as pd

from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Model

from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Conv2D, MaxPool2D, Reshape
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization, Input, Conv1D, UpSampling1D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score
from sklearn import preprocessing

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.decomposition import PCA

train = pd.read_csv("drive/My Drive/aac_spike.csv")
train.shape
examp_sub = pd.read_csv("drive/My Drive/example_sub.csv")
train = train.drop("id", axis = 1)
train = preprocessing.MaxAbsScaler().fit_transform(train)
pca = PCA(n_components = 10, svd_solver = 'randomized').fit(train)
train = pca.transform(train)

In [ ]:
train = pd.DataFrame(train)
train = train.values.reshape(144719, 10, 1)

In [ ]:
train.shape

(144719, 10, 1)

In [ ]:
input_sig = Input(batch_shape=(None,10,1))

x = Conv1D(16, 3, activation="relu", padding="same")(input_sig) # 10 dims
#x = BatchNormalization()(x)
x = MaxPooling1D(2, padding="same")(x) # 5 dims
x = Conv1D(2, 3, activation="relu", padding="same")(x) # 5 dims
#x = BatchNormalization()(x)
encoded = MaxPooling1D(2, padding="same")(x) # 3 dims

encoder = Model(input_sig, encoded)

# 3 dimensions in the encoded layer

x = Conv1D(1, 3, activation="relu", padding="same")(encoded) # 3 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 6 dims
x = Conv1D(16, 2, activation='relu')(x) # 5 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x)
x = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
x = Dense(100, activation = "relu")(x)
x = Flatten()(x)
x = Dense(10, activation = "softmax")(x)
decoded = Reshape((10, 1))(x)# 10 dims
# 10 dims
autoencoder = Model(input_sig, decoded)
autoencoder.summary()

autoencoder.compile(optimizer='sgd', loss='mse')
history = autoencoder.fit(train, train,
                epochs=100,
                batch_size=64,
                shuffle=True)



Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10, 1)             0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 10, 16)            64        
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 5, 16)             0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 5, 2)              98        
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 3, 2)              0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 3, 1)              7         
_________________________________________________________________
up_sampling1d_7 (UpSampling1 (None, 6, 1)              0   

In [ ]:
K = keras.backend
compressed_layer = 4
get_3rd_layer_output = K.function([autoencoder.layers[0].input], [autoencoder.layers[compressed_layer].output])
compressed = get_3rd_layer_output([train])[0]

np.shape(compressed)

(144719, 3, 2)

In [ ]:
representation = compressed.reshape(144719, 6)

from tensorflow.contrib.factorization.python.ops import clustering_ops
import tensorflow as tf
 
def train_input_fn():
    data = tf.constant(representation, tf.float32)
    return (data, None)
 
unsupervised_model = tf.contrib.learn.KMeansClustering(
3 #num of clusters
, distance_metric = clustering_ops.SQUARED_EUCLIDEAN_DISTANCE
) 
unsupervised_model.fit(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34e16dc4d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpgg6ull1h', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done run

KMeansClustering(params={'num_clusters': 3, 'training_initial_clusters': 'random', 'distance_metric': 'squared_euclidean', 'random_seed': 0, 'use_mini_batch': True, 'mini_batch_steps_per_iteration': 1, 'kmeans_plus_plus_num_retries': 2, 'relative_tolerance': None})

In [ ]:
from sklearn.mixture import GaussianMixture
u_mod = GaussianMixture(n_components = 3, max_iter = 250)
u_mod.fit(representation)


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=250,
                means_init=None, n_components=3, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [ ]:
u_mod.predict(representation)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
preds = unsupervised_model.predict(input_fn=train_input_fn)
predicted = []
for i in preds:
  current_cluster = i["cluster_idx"]
  predicted.append(current_cluster)
predicted

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgg6ull1h/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 2,
 1,
 1,
 1,


In [ ]:
preds = unsupervised_model.predict(input_fn=train_input_fn)
prob_list = list()
for i in preds:
  lst = []
  for j in i["all_scores"]:
    lst.append((1/j) / ( sum(1 / i["all_scores"])))
  prob_list.append(lst)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgg6ull1h/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
species1 = list()
species2 = list()
species3 = list()
for i in range(len(prob_list)):
  species1.append(prob_list[i][0])
  species2.append(prob_list[i][1])
  species3.append(prob_list[i][2])

examp_sub["species1"] = pd.Series(species1)
examp_sub["species2"] = pd.Series(species2)
examp_sub["species3"] = pd.Series(species3)

In [ ]:
examp_sub.to_csv("submission2.csv", index = False)
!cp submission2.csv "drive/My Drive/stat380_species"